In [1]:
import os
%pwd

'/media/kirti/Dev/DeepLearning/Project/E2E/ChestCancerDetection/research'

In [2]:
os.chdir('../')
%pwd

'/media/kirti/Dev/DeepLearning/Project/E2E/ChestCancerDetection'

In [3]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    training_data : Path
    params_epochs: int
    params_batch_size: int
    params_is_augmentation: bool
    params_learning_rate: float
    params_image_size: list


In [4]:
from cnnClassifier.utils.common import read_yaml, create_directories
from cnnClassifier.constants import *

import torch
import torch.nn as nn
import torchvision
from torchvision import tv_tensors
from torchsummary import summary
from torchvision.models import VGG16_Weights
from torch.utils import data
from torchvision.transforms import v2 as T
from tqdm import tqdm

In [5]:
class ConfigManager:

    def __init__(self, config_path: Path = CONFIG_FILE_PATH, 
                 params_path: Path = PARAMS_FILE_PATH):
        """        Initializes the ConfigManager with paths to the configuration and parameters files.
        Args:
            config_path (Path): Path to the configuration file.
            params_path (Path): Path to the parameters file.
        """
        self.config = read_yaml(config_path)
        self.params = read_yaml(params_path)

        create_directories([self.config.artifacts_root])

    def get_training_config(self) -> TrainingConfig:
        training = self.config.train_model
        prepare_base_model = self.config.prepare_base_model
        params = self.params
        training_data = os.path.join(self.config.data_ingestion.unzip_dir,"Data")
        create_directories(
            [Path(training.root_dir)])

        training_config = TrainingConfig(
            root_dir=Path(training.root_dir),
            trained_model_path=Path(training.model_save_path),
            updated_base_model_path=Path(prepare_base_model.update_base_model_path),
            training_data=Path(training_data),
            params_epochs=params.EPOCHS,
            params_batch_size=params.BATCH_SIZE,
            params_is_augmentation=params.AUGMENTATION,
            params_learning_rate=params.LEARNING_RATE,
            params_image_size=params.IMAGE_SIZE
        )

        return training_config


In [ ]:
class Training:
    def __init__(self, config: TrainingConfig):
        """Initializes the Training class with a configuration object.
        
        Args:
            config (TrainingConfig): Configuration object containing training parameters.
        """
        self.config = config
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    def load_base_model(self):
        """Loads the base model from the specified path."""
        if not self.config.updated_base_model_path.exists():
            raise FileNotFoundError(f"Base model not found at {self.config.updated_base_model_path}")
        
        self.model = torch.load(self.config.updated_base_model_path, map_location=self.device)
        self.model.to(self.device)

        
    def train_valid_generator(self):
        """Creates data loaders for training and validation datasets."""
        
        self.val_transform = T.Compose([
            T.Resize(self.config.params_image_size[:2]),
            T.ToTensor(),
            T.Normalize(mean=[0.485, 0.456, 0.406],
                       std=[0.229, 0.224, 0.225])
        ])

        if self.config.params_is_augmentation:
            self.train_transform = T.Compose([
                T.RandomHorizontalFlip(p=0.5),
                T.RandomRotation(degrees=(-10, 10)),
                T.ColorJitter(brightness=0.1, contrast=0.1),
                T.Resize(self.config.params_image_size[:2]),
                T.ToTensor(),
                T.Normalize(mean=[0.485, 0.456, 0.406],  # Same mean/std as ImageNet
                            std=[0.229, 0.224, 0.225])
            ])
        else:
            self.train_transform = self.val_transform

        self.train_dataset = torchvision.datasets.ImageFolder(
            root=os.path.join(self.config.training_data, "train"),
            transform=self.train_transform
        )

        self.valid_dataset = torchvision.datasets.ImageFolder(
            root=os.path.join(self.config.training_data, "valid"),
            transform=self.val_transform
        )

        self.train_loader = data.DataLoader(
            dataset=self.train_dataset,
            batch_size=self.config.params_batch_size,
            shuffle=True
        )

        self.valid_loader = data.DataLoader(
            dataset=self.valid_dataset,
            batch_size=self.config.params_batch_size,
            shuffle=False
        )

        


    def train(self):
        """Trains the model using the training dataset."""

        print(f"Training on device: {self.device}")
        print("Loading the model....")
        self.load_base_model()
        print("Model loaded successfully.")
        print("Loading Data....")
        self.train_valid_generator()
        print("Data loaded successfully.Now training started...")

        criterion = self.get_loss_function()
        optimizer = self.get_optimizer(self.model, self.config.params_learning_rate)
        scheduler = self.get_scheduler(optimizer)
        best_val_acc = 0.0
        epochs = self.config.params_epochs 

        for epoch in range(epochs):
            self.model.train()
            running_loss = 0.0
            train_accuracy = 0.0
            loop = tqdm(self.train_loader, total=len(self.train_loader))
            for images, labels in loop:
                images, labels = images.to(self.device), labels.to(self.device)
                
                optimizer.zero_grad()
                outputs = self.model(images)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()

                running_loss += loss.item() * images.size(0)
                preds = torch.argmax(outputs, dim=1)
                train_accuracy += (preds == labels).sum().item()
                loop.set_description(f"Epoch [{epoch+1}/{epochs}]")
                loop.set_postfix(loss=loss.item(), acc=train_accuracy)
                break # just to test the code, remove this line to train on all data , i used this for testing my pipeline
            epoch_loss = running_loss / len(self.train_dataset)
            epoch_accuracy = train_accuracy / len(self.train_dataset) * 100
            epoch_val_accuracy,epoch_val_loss = self.get_validation_metrics(self.model,
                                                                            self.valid_loader,
                                                                            self.device,
                                                                            len(self.valid_dataset))
            print(f"Epoch [{epoch+1}/{epochs}],\
                    Loss: {epoch_loss:.4f}, \
                    Train Accuracy: {epoch_accuracy:.3f},\
                    Validation Loss: {epoch_val_loss:.4f},\
                    Validation Accuracy: {epoch_val_accuracy:.3f}%")

            if epochs > 2: 
                scheduler.step(epoch_val_loss)
            if epoch_val_accuracy > best_val_acc:
                self.save_model(self.model,  self.config.trained_model_path.parent / f"best_model_{epoch_val_accuracy:.3f}.pth" )#os.path.join(self.config.trained_model_path.parent, "best_model.pth" ) )
                best_val_acc= epoch_val_accuracy

        self.save_model(self.model, self.config.trained_model_path)
    
    @staticmethod
    def get_loss_function():
        """Returns the loss function for training."""
        return nn.CrossEntropyLoss()
    
    @staticmethod   
    def get_optimizer(model, learning_rate):
        """Returns the optimizer for training.
        
        Args:
            model: The model to optimize.
            learning_rate (float): Learning rate for the optimizer.
        """
        print(type(model))
        return torch.optim.Adam(model.parameters(), lr=learning_rate)
    
    @staticmethod
    def get_scheduler(optimizer):
        """Returns the learning rate scheduler.
        
        Args:
            optimizer: The optimizer to schedule.
        """
        return torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.25, patience=5, verbose=True )    

    @staticmethod
    def get_validation_metrics(model, valid_loader, device,len_val_dataset):
        """Calculates the validation accuracy of the model.
        
        Args:
            model: The trained model.
            valid_loader: DataLoader for the validation dataset.
            device: Device to run the model on (CPU or GPU).
        """
        model.eval()

        accuracy = 0.0
        loss = 0.0
        with torch.no_grad():
            for images, labels in valid_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                criterion = Training.get_loss_function()
                loss += criterion(outputs, labels).item() * images.size(0)
                preds = torch.argmax(outputs, dim=1)
                accuracy += (preds == labels).sum().item()
                break # just to test the code, remove this line to validate on all data , i used this for testing my pipeline

        val_accuracy = accuracy / len_val_dataset * 100
        val_loss = loss / len_val_dataset

        return val_accuracy, val_loss
    
    
    
    @staticmethod
    def save_model(model, path):
        """Saves the trained model to the specified path.
        
        Args:
            model: The model to save.
            path (Path): Path to save the model.
        """
        torch.save(model, path)
        print(f"Model saved at {path}")


    @staticmethod
    def load_model(model_path,device):
        '''
        Loads a updated VGG16 model with customized classifier from the specified path.
        
        Args:
            model_path (str): Path to the model file.'''
        model = torch.load(model_path, map_location=device)
        return model




In [11]:
try:
    config = ConfigManager()
    training_config = config.get_training_config()
    trainer = Training(training_config)
    trainer.train()
except Exception as e:  
    raise e

[2025-07-13 14:56:40,825|(INFO)| File: common | Message: Created directory: artifacts]
[2025-07-13 14:56:40,831|(INFO)| File: common | Message: Created directory: artifacts/train_model]
Training on device: cpu
Loading the model....


/tmp/ipykernel_10534/548824563.py:16: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.model = torch.load(self.config.updated_base_model_path, map_location=self.device)
/h

Model loaded successfully.
Loading Data....
Data loaded successfully.Now training started...
<class 'torchvision.models.vgg.VGG'>


Epoch [1/1]:   0%|          | 0/77 [00:32<?, ?it/s, acc=2, loss=1.44]


Epoch [1/1],                    Loss: 0.0188,                     Train Accuracy: 0.326,                    Validation Loss: 0.0655,                    Validation Accuracy: 11.111%
Model saved at artifacts/train_model/best_model_11.11111111111111.pth
Model saved at artifacts/train_model/model.pth


In [5]:
!pip install tqdm